In [1]:
import pandas as pd
import numpy as np
import gensim
from sklearn.model_selection import train_test_split
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
from torchtext.vocab import build_vocab_from_iterator
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [323]:
text=pd.read_csv(r'C:\Users\Dell\Downloads\hate-speech-and-offensive-language-master\hate-speech-and-offensive-language-master\data\labeled_data.csv')

In [324]:
text

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [325]:
text['count'].value_counts()

3    22807
6     1571
4      211
9      167
7       27
Name: count, dtype: int64

In [326]:
test=np.where(np.logical_or((text['neither']<text['hate_speech']), (text['neither']< text['offensive_language']))&(text['class']==2))

In [327]:
data=text.iloc[test]
data['hate_speech'].value_counts()

Series([], Name: hate_speech, dtype: int64)

In [328]:
data


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet


In [329]:
text=text.drop(columns=['count','hate_speech','offensive_language','neither'])

In [330]:
text

,Unnamed: 0,class,tweet
0,0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...
24778,25291,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,1,youu got wild bitches tellin you lies


In [331]:
text['tweet']=text['tweet'].fillna('').astype(str)

TypeError: Field elements must be 2- or 3-tuples, got ''rt''

In [332]:
x=text['tweet'].copy()
y=text['class'].copy()

In [333]:
x=pd.DataFrame({'ind':x.index, 'text':x.values})


In [334]:
y

0        2
1        1
2        1
3        1
4        1
        ..
24778    1
24779    2
24780    1
24781    1
24782    2
Name: class, Length: 24783, dtype: int64

In [335]:
x

,ind,text
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
24778,24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,24779,"you've gone and broke the wrong heart baby, an..."
24780,24780,young buck wanna eat!!.. dat nigguh like I ain...
24781,24781,youu got wild bitches tellin you lies


In [336]:
x['cleaned']=x['text'].apply(lambda x: gensim.utils.simple_preprocess(x))

In [337]:
x

,ind,text,cleaned
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,"[rt, mayasolovely, as, woman, you, shouldn, co..."
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,"[rt, mleew, boy, dats, cold, tyga, dwn, bad, f..."
2,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,"[rt, urkindofbrand, dawg, rt, sbaby, life, you..."
3,3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,"[rt, c_g_anderson, viva_based, she, look, like..."
4,4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,"[rt, shenikaroberts, the, shit, you, hear, abo..."
...,...,...,...
24778,24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,"[you, muthaf, in, lie, lifeasking, corey_emanu..."
24779,24779,"you've gone and broke the wrong heart baby, an...","[you, ve, gone, and, broke, the, wrong, heart,..."
24780,24780,young buck wanna eat!!.. dat nigguh like I ain...,"[young, buck, wanna, eat, dat, nigguh, like, a..."
24781,24781,youu got wild bitches tellin you lies,"[youu, got, wild, bitches, tellin, you, lies]"


In [338]:
text="""!!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just blows me..claim you so faithful and down for somebody but still fucking with hoes! &#128514;&#128514;&#128514;"
"""
def preprocess_text(text):
    text=text.lower()
    text=re.sub(r'\d+', '',text)
    text=re.sub(r'[^\w\s]', '',text)
    tokens=nltk.word_tokenize(text)
    return tokens

In [339]:
def remove_stopwords(tokens):
    stop_words=set(stopwords.words('english'))
    filtered_tokens=[word for word in tokens if word not in stop_words]
    return filtered_tokens
    

In [340]:
def lemmatization(tokens):
    lemmatizer=nltk.WordNetLemmatizer()
    lemmatized_tokens=[lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

In [341]:
def clean_text(text):
    tokens=preprocess_text(text)
    filtered_tokens=remove_stopwords(tokens)
    lemmatized_tokens=lemmatization(filtered_tokens)
    #clean_text=''.join(lemmatized_tokens)
    return filtered_tokens

In [342]:
lemmatized_tokens

['t_madison_x',
 'shit',
 'blow',
 'meclaim',
 'faithful',
 'somebody',
 'still',
 'fucking',
 'hoe']

In [343]:
x['cleaned2']=x['text'].apply(lambda x: clean_text(x))

In [344]:
x

,ind,text,cleaned,cleaned2
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,"[rt, mayasolovely, as, woman, you, shouldn, co...","[rt, mayasolovely, woman, shouldnt, complain, ..."
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,"[rt, mleew, boy, dats, cold, tyga, dwn, bad, f...","[rt, mleew, boy, dats, coldtyga, dwn, bad, cuf..."
2,2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,"[rt, urkindofbrand, dawg, rt, sbaby, life, you...","[rt, urkindofbrand, dawg, rt, sbabylife, ever,..."
3,3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,"[rt, c_g_anderson, viva_based, she, look, like...","[rt, c_g_anderson, viva_based, look, like, tra..."
4,4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,"[rt, shenikaroberts, the, shit, you, hear, abo...","[rt, shenikaroberts, shit, hear, might, true, ..."
...,...,...,...,...
24778,24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,"[you, muthaf, in, lie, lifeasking, corey_emanu...","[yous, muthafin, lie, lifeasking, _pearls, cor..."
24779,24779,"you've gone and broke the wrong heart baby, an...","[you, ve, gone, and, broke, the, wrong, heart,...","[youve, gone, broke, wrong, heart, baby, drove..."
24780,24780,young buck wanna eat!!.. dat nigguh like I ain...,"[young, buck, wanna, eat, dat, nigguh, like, a...","[young, buck, wan, na, eat, dat, nigguh, like,..."
24781,24781,youu got wild bitches tellin you lies,"[youu, got, wild, bitches, tellin, you, lies]","[youu, got, wild, bitches, tellin, lies]"


In [346]:
xtrain1,xtest1,ytrain1,ytest1=train_test_split (x['cleaned'], y , test_size=0.2)
xtest1,xval1,ytest1,yval1=train_test_split (xtest1, ytest1 , test_size=0.5)

In [347]:
w2v_model = gensim.models.Word2Vec(xtrain1,
                                   vector_size=100,
                                   window=5,
                                   min_count=10,max_vocab_size=32000)
#

In [348]:
w2v_model.wv.most_similar('dirty')

[('sweet', 0.9975654482841492),
 ('free', 0.9975312948226929),
 ('coming', 0.9974188804626465),
 ('double', 0.9973883032798767),
 ('king', 0.9973591566085815),
 ('ran', 0.9972456693649292),
 ('short', 0.9971978664398193),
 ('star', 0.9971783757209778),
 ('fries', 0.9971531629562378),
 ('instagram', 0.9971056580543518)]

In [349]:
doc=list(x['cleaned'])
st=[]
for ele in doc:
    st=st+ele

In [350]:
st=[st]

In [351]:
sum=0
for ele in doc:
    sum+=len(ele)
sum

330043

In [352]:
len(st)

1

In [353]:
w2v_model = gensim.models.word2vec.Word2Vec(vector_size=100, window=5, min_count=2)
w2v_model.build_vocab(st)

vocab_size = len(w2v_model.wv)
print("Vocab size", vocab_size)

Vocab size 12500


In [354]:
w2v_model.train(st, total_examples=len(st), epochs=100)

(1000000, 33004300)

In [356]:
vocab = build_vocab_from_iterator(st, min_freq=2)

In [362]:
type(vocab)

torchtext.vocab.vocab.Vocab

In [368]:
w2v_model.save(r'C:\Users\Dell\Documents\projects\hate speech detection\hate-speech-and-offensive-language-master\w2v_embeddings1-100')

In [2]:
loaded_model = KeyedVectors.load(r'C:\\Users\\Dell\\Documents\\projects\\hate speech detection\\hate-speech-and-offensive-language-master\\w2v_embeddings1-100', binary=True)

TypeError: SaveLoad.load() got an unexpected keyword argument 'binary'

In [3]:
model1 = Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, norm_only=True)

AttributeError: type object 'Word2Vec' has no attribute 'load_word2vec_format'

In [377]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
C:\Users\Dell/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz
